# Create a Grafana Data Source and Dashboard

Follow the steps in this this tutorial to define a custom "iguazio" Grafana data source and create a "stocks" Grafana dashboard that uses this data source to query tables in the Iguazio Data Science Platform (**"the platform"**).

In [1]:
!pip install git+https://github.com/v3io/grafwiz.git

  Cloning https://github.com/v3io/grafwiz.git to /tmp/pip-req-build-fwkh1tj4
  Running setup.py bdist_wheel for grafwiz ... done
  Stored in directory: /tmp/pip-ephem-wheel-cache-0jnpcojn/wheels/8f/07/f3/185d8b497806aae5db86574055449669bd6c6658be2a5f9ec4
Successfully built grafwiz


In [2]:
from grafwiz import *
import os

In [3]:
# Grafana internal cluster address (will be http://grafana)
grafana_url = 'http://grafana'
# External URL of the demo's stream-viewer Nuclio serverless function
# Replace the string after the app. with the dashboard URL (e.g. stream-view.iguazio.app.product4.iguazio-cd2.com)
streamview_url = 'https://stream-view.iguazio.app.<Dashboard URL string>/'

In [4]:
# Create a new Grafana dashboard
dash = Dashboard("stocks", start='now-1d', dataSource='iguazio')

# Add a symbol combo box (template) with data from the stocks table
dash.template(name="SYMBOL", label="Symbol", query="fields=symbol;table=stocks;backend=kv;container=bigdata")

# Create a table and log viewer in one row
tbl = Table('tbl1',span=8).source(table='stocks',fields=['symbol','name','currency','price','last_trade','timezone','exchange'],container='bigdata')
log = Ajax(title='Log',url=streamview_url)
dash.row([tbl,log])

# Create 3 charts on the second row
dash.row([Graph(metric).series(table="stock_metrics", fields=[metric], filter='symbol=="$SYMBOL"',container='bigdata') for metric in ['price','volume','sentiment']])

# Deploy to Grafana
dash.deploy(grafana_url)

Dashboard stocks created successfully
